AAAAAAAAAAAAAAAAAAAAAAAAAAA




MY MATH IS WRONGITHWREGPI@ERJW"
**From**

   $$
     a(x-n)^{2}+b(y-m)^{2}+cxy \;=\; d .
   $$


**Solve for $c$**

$$
  \boxed{\,c=\dfrac{\,d-a(x-n)^{2}-b(y-m)^{2}\,}{xy}\,}
  \qquad\bigl(x\neq 0,\;y\neq 0\bigr).
$$

Showing $c$ equals the residual uniquely whenever $xy$ is non‑zero; otherwise, the $c$ term implodes as the $xy$ denominator vanishes and further information about the specific conic or application is needed.



Does each locat
 __ __
|[][]
|[][]


 n1, m1
 n2, m2


In [ ]:
import imageio
import numpy as np
from scikit-image.draw import ellipse
import cv2
import time
import math


In [1]:
def generate_uint8_labels(w, h, cells_data, use_vectorized=True):
    """
    Parameters:
    - w, h: image width and height
    - cells_data: dict with keys:
        - 'indices': list of cell IDs [1, 2, 3, ...]
        - 'fluorescence': list of fluorescence levels (not used for segmentation)
        - 'size': list of cell diameters [d1, d2, ...] (optional, can use shape instead)
        - 'shape': list of (a,b) tuples for ellipse semi-axes
        - 'location': list of (n,m) tuples for cell centers in TOP-LEFT coordinates
        - 'rotation': list of rotation in degrees [r1, r2, ...]
    - use_vectorized: if True, use fast vectorized method; if False, use mathematical pixel-by-pixel

    Returns:
    - uint8_labels: np.ndarray of shape (h, w) with dtype=np.uint8
    """
    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices = cells_data["indices"]
    shapes = cells_data["shape"]
    locations = cells_data["location"]
    rotations = cells_data["rotation"]

    print(f"Generating uint8 labels for {len(indices)} cells...")
    print(f"Output array shape: {uint8_labels.shape}, dtype: {uint8_labels.dtype}")
    print(f"Using {'vectorized' if use_vectorized else 'mathematical'} ellipse generation")

    #mask generation method
    mask_function = create_ellipse_mask_vectorized if use_vectorized else create_ellipse_mask_mathematical

    for cell_id, (semi_a, semi_b), (center_x, center_y), rot_numbers in zip(indices, shapes, locations, rotations):
        if cell_id > 255:
            raise ValueError(f"Cell ID {cell_id} exceeds uint8 maximum of 255")
            coeffs = ellipse_params_to_general_form(center_x, center_y, semi_a, semi_b, angle_deg)
        cell_mask = mask_function(w, h, coeffs)
        uint8_labels[cell_mask] = cell_id

        pixel_count = np.sum(cell_mask)
        print(f"  Cell {cell_id}: center=({center_x},{center_y}), shape=({semi_a},{semi_b}), "
              f"rotation={rot_numbers} number, pixels={pixel_count}")

    unique_labels = np.unique(uint8_labels)
    print(f"\nUint8 label summary:")
    print(f"  Unique values: {unique_labels}")
    print(f"  Background pixels (0): {np.sum(uint8_labels == 0)}")
    print(f"  Total labeled pixels: {np.sum(uint8_labels > 0)}")

    return uint8_labels

def ellipse_params_to_general_form(center_x: float,
                                   center_y: float,
                                   semi_a: float,
                                   semi_b: float,
                                   angle_deg: float,
                                   *,
                                   circle_rel_eps: float | None = None):
    """
    Parameters
    ----------
    - center_x, center_y : float
        - Ellipse centre (h, k).
    - semi_a, semi_b : float
        - Semi‑major / semi‑minor axes *in pixels* (must be > 0).
    - angle_deg : float
        - Rotation angle θ in degrees.
    - circle_rel_eps : float or None, optional
        - Relative tolerance for deciding the ellipse is a circle.  If *None*
        (default), we use 2*machine ε(≈4.4e-16).  A value of, say,  1e‑8 duplicates the common graphics‑library threshold.
    """
    if semi_a <= 0 or semi_b <= 0:
        raise ValueError("semi_a and semi_b must be positive")
    """
    VERY NUMERICALLY CRAZY BAD
        theta = np.deg2rad(angle_deg%360)
        cos_t = np.cos(theta)
        sin_t = np.sin(theta)

        a_coeff = (cos_t / semi_a) ** 2 + (sin_t / semi_b) ** 2
        b_coeff = (sin_t / semi_a) ** 2 + (cos_t / semi_b) ** 2
    """
    scale = max(semi_a, semi_b)
    a_hat, b_hat = semi_a / scale, semi_b / scale   #to bound 0 < a_hat, b_hat ≤ 1

    #b_hat or a_hat just means "normalized and i could be 1 or smaller"
    inv_a2 = 1.0 / (a_hat * a_hat)
    inv_b2 = 1.0 / (b_hat * b_hat)

    #c_coeff = 2 * sin_t * cos_t * (1/semi_a**2 - 1/semi_b**2)
    #init used this one but there is cancellation error in the parentheses so i made
    # delta, inv1 and inv2 for numerical stabilityinv_a2 = 1.0 / (a*a)
    #delta used later

    theta = np.deg2rad(angle_deg % 360)
    sin_t, cos_t = math.sin(theta), math.cos(theta)

    theta  = math.radians(angle_deg % 360.0)
    sin_t  = math.sin(theta)
    cos_t  = math.cos(theta)

    # ------------------------------------------------------------------
    # 3Stable product for Δ = 1/a² − 1/b²  (avoids catastrophic cancellation)
    #     Δ = (b_hat − a_hat)(b_hat + a_hat) / (a_hat² b_hat²)
    #     Underflow safeguard: if Δ under‑flows to zero at float64, we can
    #     safely snap B to zero – numerically, the ellipse is essentially
    #     axis‑aligned.
    diff   = (b_hat - a_hat) * (b_hat + a_hat)
    denom  = (a_hat * a_hat) * (b_hat * b_hat)
    delta  = diff / denom if abs(diff) > _DOUBLE_TINY else 0.0
    1/semi_a**2 - 1/semi_b**2

    sc = math.fma(sin_t, cos_t, 0.0) if hasattr(math, "fma") else sin_t * cos_t
    B  = 2.0 * sc * delta

    A  = (cos_t * cos_t) * inv_a2 + (sin_t * sin_t) * inv_b2
    C  = (sin_t * sin_t) * inv_a2 + (cos_t * cos_t) * inv_b2

    #undo rescaling - coeff for original pixel units
    scale2 = scale * scale
    A     /= scale2
    B     /= scale2
    C     /= scale2

    return {
        "a": A,
        "b": C,
        "c": B,
        "d": 1.0,
        "h": center_x,
        "k": center_y,
    }

def create_ellipse_mask_mathematical(w, h, cx, cy, a, b, angle_deg):

    angle_rad = np.deg2rad(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    mask = np.zeros((h, w), dtype=bool)

    for y in range(h):
        for x in range(w):

            x_rel = x - cx
            y_rel = y - cy


            x_rot = cos_a * x_rel + sin_a * y_rel
            y_rot = -sin_a * x_rel + cos_a * y_rel

            # Check if rotated point is inside standard ellipse equation
            # (x/a)² + (y/b)² ≤ 1
            if (x_rot/a)**2 + (y_rot/b)**2 <= 1:
                mask[y, x] = True

    return mask


def create_ellipse_mask_vectorized(w, h, cx, cy, a, b, angle_deg):

    #create coordinate grids
    y_grid, x_grid = np.ogrid[:h, :w]

    #translate all coordinates to ellipse-centered system
    x_centered = x_grid - cx
    y_centered = y_grid - cy

    #rotate all points
    angle_rad = np.deg2rad(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    x_rotated = cos_a * x_centered + sin_a * y_centered
    y_rotated = -sin_a * x_centered + cos_a * y_centered

    ellipse_values = (x_rotated/a)**2 + (y_rotated/b)**2
    mask = ellipse_values <= 1

    return mask


def save_uint8_labels(uint8_labels, base_filename):

    uint8_labels.tofile(f"{base_filename}.uint8")
    print(f"Saved raw binary: {base_filename}.uint8")

    np.save(f"{base_filename}.npy", uint8_labels)
    print(f"Saved NumPy array: {base_filename}.npy")

    imageio.imwrite(f"{base_filename}_labels.png", uint8_labels)
    print(f"Saved PNG: {base_filename}_labels.png")

    import json
    metadata = {
        'width': uint8_labels.shape[1],
        'height': uint8_labels.shape[0],
        'dtype': 'uint8',
        'unique_labels': np.unique(uint8_labels).tolist(),
        'max_label': int(uint8_labels.max())
    }
    with open(f"{base_filename}_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved metadata: {base_filename}_metadata.json")


def load_uint8_labels(filename, width, height):

    flat_array = np.fromfile(filename, dtype=np.uint8)

    uint8_labels = flat_array.reshape((height, width))

    return uint8_labels


def visualize_uint8_labels(uint8_labels, colormap=None):
    if colormap is None:
        colormap = np.array([
            [70, 50, 160],   # 0: Background (purple)
            [100, 200, 255], # 1: Cell 1 (light blue)
            [80, 180, 255],  # 2: Cell 2 (blue)
            [120, 255, 255], # 3: Cell 3 (cyan)
            [100, 200, 100], # 4: Cell 4 (dark green)
            [150, 255, 150], # 5: Cell 5 (light green)
            [255, 255, 100], # 6: Cell 6 (yellow)
            [255, 200, 100], # 7: Cell 7 (orange)
            [255, 150, 150], # 8: Cell 8 (pink)
            [200, 150, 255], # 9: Cell 9 (light purple)
        ], dtype=np.uint8)

    max_label = uint8_labels.max()
    if max_label >= len(colormap):
        n_extra = max_label + 1 - len(colormap)
        extra_colors = np.random.randint(50, 256, (n_extra, 3), dtype=np.uint8)
        colormap = np.vstack([colormap, extra_colors])

    rgb_image = colormap[uint8_labels]

    return rgb_image


def compare_mask_generation_methods():

    w, h = 128, 128
    cx, cy = 64, 64
    a, b = 20, 15
    angle = 30

    print("=== Comparing Ellipse Mask Generation Methods ===")
    print(f"Image size: {w}x{h}")
    print(f"Ellipse: center=({cx},{cy}), axes=({a},{b}), rotation={angle}°")

    # Mathematical method (pixel-by-pixel)
    start = time.time()
    mask_math = create_ellipse_mask_mathematical(w, h, cx, cy, a, b, angle)
    time_math = time.time() - start
    pixels_math = np.sum(mask_math)

    # Vectorized method
    start = time.time()
    mask_vec = create_ellipse_mask_vectorized(w, h, cx, cy, a, b, angle)
    time_vec = time.time() - start
    pixels_vec = np.sum(mask_vec)

    # Compare results
    print(f"\nMathematical method:")
    print(f"  Time: {time_math*1000:.2f} ms")
    print(f"  Pixels in ellipse: {pixels_math}")

    print(f"\nVectorized method:")
    print(f"  Time: {time_vec*1000:.2f} ms")
    print(f"  Pixels in ellipse: {pixels_vec}")
    print(f"  Speedup: {time_math/time_vec:.1f}x faster")

    # Verify they produce identical results
    identical = np.array_equal(mask_math, mask_vec)
    print(f"\nMasks identical: {identical}")

    if not identical:
        diff = np.sum(mask_math != mask_vec)
        print(f"  Differences: {diff} pixels")


def compare_mask_generation_methods():


    # Test parameters
    w, h = 128, 128
    cx, cy = 64, 64
    a, b = 20, 15
    angle = 30

    print("=== Comparing Ellipse Mask Generation Methods ===")
    print(f"Image size: {w}x{h}")
    print(f"Ellipse: center=({cx},{cy}), axes=({a},{b}), rotation={angle}°")

    # Mathematical method (pixel-by-pixel)
    start = time.time()
    mask_math = create_ellipse_mask_mathematical(w, h, cx, cy, a, b, angle)
    time_math = time.time() - start
    pixels_math = np.sum(mask_math)

    # Vectorized method
    start = time.time()
    mask_vec = create_ellipse_mask_vectorized(w, h, cx, cy, a, b, angle)
    time_vec = time.time() - start
    pixels_vec = np.sum(mask_vec)

    # Compare results
    print(f"\nMathematical method:")
    print(f"  Time: {time_math*1000:.2f} ms")
    print(f"  Pixels in ellipse: {pixels_math}")

    print(f"\nVectorized method:")
    print(f"  Time: {time_vec*1000:.2f} ms")
    print(f"  Pixels in ellipse: {pixels_vec}")
    print(f"  Speedup: {time_math/time_vec:.1f}x faster")

    # Verify they produce identical results
    identical = np.array_equal(mask_math, mask_vec)
    print(f"\nMasks identical: {identical}")

    if not identical:
        diff = np.sum(mask_math != mask_vec)
        print(f"  Differences: {diff} pixels")


def example_complete_pipeline():
    w, h = 128, 128

    cells_data = {
        'indices': [1, 2, 3, 4, 5, 6, 7, 8, 9],
        'fluorescence': [100, 120, 80, 150, 90, 110, 130, 140, 95],
        'size': [15, 18, 14, 20, 16, 17, 19, 18, 15],
        'shape': [(8, 7), (10, 9), (7, 7), (11, 10), (9, 8),
                  (9, 9), (10, 8), (9, 9), (8, 7)],
        'location': [(30, 25), (30, 50), (40, 80), (60, 30),
                     (85, 70), (110, 25), (110, 85), (110, 110), (80, 110)],
        'rotation': [0, 15, -20, 30, 0, 45, -10, 0, 25]
    }

    print("=== STEP 1: Generate uint8 labels ===")

    uint8_labels = generate_uint8_labels(w, h, cells_data, use_vectorized=True)

    print("\n=== Optional: Generate with mathematical method for comparison ===")
    uint8_labels_math = generate_uint8_labels(w, h, cells_data, use_vectorized=False)
    print(f"Results identical: {np.array_equal(uint8_labels, uint8_labels_math)}")

    print("\n=== STEP 2: Save uint8 labels ===")
    save_uint8_labels(uint8_labels, "yeast_segmentation")

    print("\n=== STEP 3: Visualize (optional) ===")

    rgb_visualization = visualize_uint8_labels(uint8_labels)
    print(f"RGB visualization shape: {rgb_visualization.shape}")

    imageio.imwrite("yeast_segmentation_visual.png", rgb_visualization)
    print("Saved visualization: yeast_segmentation_visual.png")

    return uint8_labels, rgb_visualization


def inspect_uint8_output(uint8_labels):

    print("=== Uint8 Label Array Inspection ===")
    print(f"Shape: {uint8_labels.shape}")
    print(f"Data type: {uint8_labels.dtype}")
    print(f"Memory usage: {uint8_labels.nbytes} bytes")
    print(f"Min value: {uint8_labels.min()}")
    print(f"Max value: {uint8_labels.max()}")

    # Count pixels per label
    unique, counts = np.unique(uint8_labels, return_counts=True)
    print("\nPixel counts per label:")
    for label, count in zip(unique, counts):
        if label == 0:
            print(f"  Background: {count} pixels")
        else:
            print(f"  Cell {label}: {count} pixels")

    # Show a small sample of the raw values
    print("\nSample of raw uint8 values (top-left 10x10):")
    print(uint8_labels[:10, :10])


#optimized
def generate_uint8_labels_cv2(w, h, cells_data):

    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices = cells_data['indices']
    shapes = cells_data['shape']
    locations = cells_data['location']
    rotations = cells_data['rotation']

    for cell_id, (a, b), (cx, cy), angle in zip(indices, shapes, locations, rotations):
        #cv2 ellipse parameters
        center = (int(cx), int(cy))
        axes = (int(a), int(b))

        #draw filled ellipse with cell ID as the fill value
        cv2.ellipse(uint8_labels, center, axes, angle, 0, 360, int(cell_id), -1)

    return uint8_labels

In [ ]:
def generate_uint8_labels_cv2(w, h, cells_data):
    #optimized
    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices = cells_data['indices']
    shapes = cells_data['shape']
    locations = cells_data['location']
    rotations = cells_data['rotation']

    for cell_id, (a, b), (cx, cy), angle in zip(indices, shapes, locations, rotations):
        center = (int(cx), int(cy))
        axes = (int(a), int(b))

        cv2.ellipse(uint8_labels, center, axes, angle, 0, 360, int(cell_id), -1)

    return uint8_labels